In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import brickschema
import datetime
from relaxation_graphs.fast_relaxation_graph import *
from evaluation.similarity_score import *
import uuid
import pickle
from rdflib import URIRef, Literal, BNode
from queries.mortar import queries as q1
%matplotlib inline

In [2]:
font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 32}

matplotlib.rc('font', **font)

In [3]:
file = "Data - Brick models/acad.ttl"

running_example = """SELECT * WHERE {
 ?sen   rdf:type   brick:Supply_Air_Temperature_Sensor .
 ?sp    rdf:type   brick:Supply_Air_Temperature_Setpoint .
 ?ahu   rdf:type   brick:AHU .
 ?ahu   brick:feeds ?thing .
 ?thing  brick:hasPoint ?sen .
 ?thing  brick:hasPoint ?sp .
}"""


## max_level = 3

In [4]:
max_level = 3
print_counter = 3
G = get_fast_relaxed_graph(query=running_example, max_level=max_level)
already_done_queries = []

In [5]:
g = brickschema.Graph(load_brick=True)
g.load_file(file)
g.expand(profile="owlrl")
select_statement = running_example.split("{")[0] + "{\n"

In [6]:
total_instances = get_total_num_instances(g, 'brick:Equipment') + get_total_num_instances(g, 'brick:Location') + get_total_num_instances(g, 'brick:Point')
total_instances

1920

In [7]:
original_uuid = list(G.nodes())[0]
original_query = G.nodes()[original_uuid]['query']

### case 1: all weights=1

In [8]:
scores = {}
for idx, node in G.nodes().data():    
    relaxed_query = list(node['query'])
    score = sim_score(original_query, relaxed_query, g, total_instances)
    
    level = node['level']
    if score in scores:
        scores[score].append(relaxed_query)
    else:
        scores[score] = [relaxed_query]

In [9]:
for score in sorted(list(scores.keys()), reverse = True):
    print("score: {}, number of relaxed queries with this score: {}".format(score, len(scores[score])))

score: 1, number of relaxed queries with this score: 9
score: 0.8703953925243878, number of relaxed queries with this score: 7
score: 0.8691185533597107, number of relaxed queries with this score: 4
score: 0.8193389185022003, number of relaxed queries with this score: 11
score: 0.8017951543953568, number of relaxed queries with this score: 1
score: 0.7816289042500114, number of relaxed queries with this score: 1
score: 0.7681412256571158, number of relaxed queries with this score: 11
score: 0.71314881958023, number of relaxed queries with this score: 5
score: 0.7121026555599422, number of relaxed queries with this score: 1
score: 0.6685865836199897, number of relaxed queries with this score: 5
score: 0.6676057908190676, number of relaxed queries with this score: 1
score: 0.6617400035965336, number of relaxed queries with this score: 1
score: 0.6293680010868559, number of relaxed queries with this score: 6
score: 0.5477990083482832, number of relaxed queries with this score: 1
score: 0,

In [10]:
results = {}

In [11]:
max_score = 0
count = 0
queries_to_print1 = []

for score in sorted(list(scores.keys()), reverse = True):
    if score > max_score:
        max_score = score

    for i in scores[score]:
        if not str(i) in results:
            brick_query = generate_brick_query_from_node(i, select_statement=select_statement)
            res = run_brick_query(building_model=g, query=brick_query)
            results[str(i)] = res
        else:
            res = results[str(i)]
            
        if len(res) > 0:
            if not (i, score) in queries_to_print1:
                queries_to_print1.append((i, score))
            print(res)
        print(i, score)

    count = len(queries_to_print1)
    
    if count >= print_counter:
        break

for q, sc in queries_to_print1:
    print(q, sc)

[['?sen', 'rdf:type', 'brick:Supply_Air_Temperature_Sensor'], ['?sp', 'rdf:type', 'brick:Supply_Air_Temperature_Setpoint'], ['?ahu', 'rdf:type', 'brick:AHU'], ['?ahu', 'brick:feeds', '?thing'], ['?thing', 'brick:hasPoint', '?sen'], ['?thing', 'brick:hasPoint', '?sp']] 1
[['?sen', 'rdf:type', 'brick:Supply_Air_Temperature_Sensor'], ['?sp', 'rdf:type', 'brick:Supply_Air_Temperature_Setpoint'], ['?ahu', 'rdf:type', 'brick:AHU'], ['?ahu', 'brick:feeds', '?thing'], ['?thing', 'brick:hasPoint', '?sen'], ['?thing', 'brick:hasPoint', '?sp']] 1
[['?sen', 'rdf:type', 'brick:Supply_Air_Temperature_Sensor'], ['?sp', 'rdf:type', 'brick:Supply_Air_Temperature_Setpoint'], ['?ahu', 'rdf:type', 'brick:AHU'], ['?ahu', 'brick:feeds', '?thing'], ['?thing', 'brick:hasPoint', '?sen'], ['?thing', 'brick:hasPoint+', '?sp']] 1
[['?sen', 'rdf:type', 'brick:Supply_Air_Temperature_Sensor'], ['?sp', 'rdf:type', 'brick:Supply_Air_Temperature_Setpoint'], ['?ahu', 'rdf:type', 'brick:AHU'], ['?ahu', 'brick:feeds', '?t

#### case 2: triple 1,2,3,4 weights = 0.5, triple 5 weight = 1

In [12]:
scores2 = {}
for idx, node in G.nodes().data():    
    relaxed_query = list(node['query'])
    score2 = sim_score(original_query, relaxed_query, g, total_instances)
    
    if relaxed_query[0][2] == 'brick:Supply_Air_Temperature_Sensor': #1
        score2 = score2*0.5
    if relaxed_query[1][2] == 'brick:Supply_Air_Temperature_Setpoint': #2
        score2 = score2*0.5
    if relaxed_query[2][2] == 'brick:AHU': #3
        score2 = score2*0.5
    if relaxed_query[3][1] == 'brick:feeds': #4
        score2 = score2*0.5
    
    if score2 in scores2:
        scores2[score2].append(relaxed_query)
    else:
        scores2[score2] = [relaxed_query]

In [13]:
for score2 in sorted(list(scores2.keys()), reverse = True):
    print("score: {}, number of relaxed queries with this score: {}".format(score2, len(scores2[score2])))

score: 0.356574409790115, number of relaxed queries with this score: 1
score: 0.33429329180999484, number of relaxed queries with this score: 1
score: 0.31468400054342793, number of relaxed queries with this score: 1
score: 0.2738995041741416, number of relaxed queries with this score: 1
score: 0.21759884813109695, number of relaxed queries with this score: 3
score: 0.21727963833992767, number of relaxed queries with this score: 1
score: 0.20483472962555008, number of relaxed queries with this score: 4
score: 0.19203530641427896, number of relaxed queries with this score: 4
score: 0.1782872048950575, number of relaxed queries with this score: 4
score: 0.17802566388998556, number of relaxed queries with this score: 1
score: 0.16714664590499742, number of relaxed queries with this score: 4
score: 0.1669014477047669, number of relaxed queries with this score: 1
score: 0.15734200027171397, number of relaxed queries with this score: 5
score: 0.125, number of relaxed queries with this score:

In [14]:
max_score2 = 0
count = 0
queries_to_print2 = []

for score2 in sorted(list(scores2.keys()), reverse = True):
    if score2 > max_score2:
        max_score2 = score2

    for i in scores2[score2]:
        if not str(i) in results:
            brick_query = generate_brick_query_from_node(i, select_statement=select_statement)
            res = run_brick_query(building_model=g, query=brick_query)
            results[str(i)] = res
        else:
            res = results[str(i)]

        if len(res) > 0:
            if not (i, score2) in queries_to_print2:
                queries_to_print2.append((i, score2))
            print(res)
        print(i, score2)
    count = len(queries_to_print2)
    
    if count >= print_counter:
        break

for q, sc in queries_to_print2:
    print(q, sc)

[rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1283N'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM2200C'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1283A'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM2307A'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM2328'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM2307'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM2200C'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM2306'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1283S'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1283K'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1283E'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1283G'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1283S'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1283Q'), rdflib.term.URIRef('ht

### case 3: triple 1,2, weights = 0.1, triples 3,4 weights = 0.5, triple 5 weight = 1

In [15]:
scores3 = {}
for idx, node in G.nodes().data():    
    relaxed_query = list(node['query'])
    score3 = sim_score(original_query, relaxed_query, g, total_instances)
    
    if relaxed_query[0][2] == 'brick:Supply_Air_Temperature_Sensor': #1
        score3 = score3*0.1
    if relaxed_query[1][2] == 'brick:Supply_Air_Temperature_Setpoint': #2
        score3 = score3*0.1
    if relaxed_query[2][2] == 'brick:AHU': #3
        score3 = score3*0.5
    if relaxed_query[3][1] == 'brick:feeds': #4
        score3 = score3*0.5
    
    if score3 in scores3:
        scores3[score3].append(relaxed_query)
    else:
        scores3[score3] = [relaxed_query]

In [16]:
for score3 in sorted(list(scores3.keys()), reverse = True):
    print("score: {}, number of relaxed queries with this score: {}".format(score3, len(scores3[score3])))

score: 0.356574409790115, number of relaxed queries with this score: 1
score: 0.2738995041741416, number of relaxed queries with this score: 1
score: 0.1782872048950575, number of relaxed queries with this score: 4
score: 0.17802566388998556, number of relaxed queries with this score: 1
score: 0.06685865836199897, number of relaxed queries with this score: 1
score: 0.06293680010868559, number of relaxed queries with this score: 1
score: 0.043519769626219396, number of relaxed queries with this score: 3
score: 0.04345592766798553, number of relaxed queries with this score: 1
score: 0.04096694592511002, number of relaxed queries with this score: 4
score: 0.03342932918099949, number of relaxed queries with this score: 4
score: 0.03338028954095338, number of relaxed queries with this score: 1
score: 0.031468400054342795, number of relaxed queries with this score: 5
score: 0.021759884813109698, number of relaxed queries with this score: 4
score: 0.021727963833992767, number of relaxed queri

In [17]:
max_score3 = 0
count = 0
queries_to_print3 = []

for score3 in sorted(list(scores3.keys()), reverse = True):
    if score3 > max_score3:
        max_score3 = score3

    for i in scores3[score3]:
        if not str(i) in results:
            brick_query = generate_brick_query_from_node(i, select_statement=select_statement)
            res = run_brick_query(building_model=g, query=brick_query)
            results[str(i)] = res
        else:
            res = results[str(i)]

        if len(res) > 0:
            if not (i, score3) in queries_to_print3:
                queries_to_print3.append((i, score3))
            print(res)
        print(i, score3)
    count = len(queries_to_print3)
    
    if count >= print_counter:
        break

for q, sc in queries_to_print3:
    print(q, sc)

[rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1283N'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM2200C'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1283A'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM2307A'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM2328'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM2307'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM2200C'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM2306'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1283S'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1283K'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1283E'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1283G'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1283S'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1283Q'), rdflib.term.URIRef('ht

In [18]:
results

{"[['?sen', 'rdf:type', 'brick:Supply_Air_Temperature_Sensor'], ['?sp', 'rdf:type', 'brick:Supply_Air_Temperature_Setpoint'], ['?ahu', 'rdf:type', 'brick:AHU'], ['?ahu', 'brick:feeds', '?thing'], ['?thing', 'brick:hasPoint', '?sen'], ['?thing', 'brick:hasPoint', '?sp']]": [],
 "[['?sen', 'rdf:type', 'brick:Supply_Air_Temperature_Sensor'], ['?sp', 'rdf:type', 'brick:Supply_Air_Temperature_Setpoint'], ['?ahu', 'rdf:type', 'brick:AHU'], ['?ahu', 'brick:feeds', '?thing'], ['?thing', 'brick:hasPoint', '?sen'], ['?thing', 'brick:hasPoint+', '?sp']]": [],
 "[['?sen', 'rdf:type', 'brick:Supply_Air_Temperature_Sensor'], ['?sp', 'rdf:type', 'brick:Supply_Air_Temperature_Setpoint'], ['?ahu', 'rdf:type', 'brick:AHU'], ['?ahu', 'brick:feeds', '?thing'], ['?thing', 'brick:hasPoint+', '?sen'], ['?thing', 'brick:hasPoint', '?sp']]": [],
 "[['?sen', 'rdf:type', 'brick:Supply_Air_Temperature_Sensor'], ['?sp', 'rdf:type', 'brick:Supply_Air_Temperature_Setpoint'], ['?ahu', 'rdf:type', 'brick:AHU'], ['?ahu

In [19]:
queries_to_print1

[([['?sen', 'rdf:type', 'brick:Supply_Air_Temperature_Sensor'],
   ['?sp', 'rdf:type', 'brick:Air_Temperature_Setpoint'],
   ['?ahu', 'rdf:type', 'brick:AHU'],
   ['?ahu', 'brick:feeds', '?thing'],
   ['?thing', 'brick:hasPoint', '?sen'],
   ['?thing', 'brick:hasPoint', '?sp']],
  0.8193389185022003),
 ([['?sen', 'rdf:type', 'brick:Supply_Air_Temperature_Sensor'],
   ['?sp', 'rdf:type', 'brick:Air_Temperature_Setpoint'],
   ['?ahu', 'rdf:type', 'brick:AHU'],
   ['?ahu', 'brick:feeds', '?thing'],
   ['?thing', 'brick:hasPoint', '?sen'],
   ['?thing', 'brick:hasPoint+', '?sp']],
  0.8193389185022003),
 ([['?sen', 'rdf:type', 'brick:Supply_Air_Temperature_Sensor'],
   ['?sp', 'rdf:type', 'brick:Air_Temperature_Setpoint'],
   ['?ahu', 'rdf:type', 'brick:AHU'],
   ['?ahu', 'brick:feeds', '?thing'],
   ['?thing', 'brick:hasPoint+', '?sen'],
   ['?thing', 'brick:hasPoint', '?sp']],
  0.8193389185022003),
 ([['?sen', 'rdf:type', 'brick:Supply_Air_Temperature_Sensor'],
   ['?sp', 'rdf:type', 'b

In [20]:
queries_to_print2

[([['?sen', 'rdf:type', 'brick:Air_Temperature_Sensor'],
   ['?sp', 'rdf:type', 'brick:Air_Temperature_Setpoint'],
   ['?ahu', 'rdf:type', 'brick:AHU'],
   ['?ahu', 'brick:feeds+', '?thing'],
   ['?thing', 'brick:hasPoint', '?sen'],
   ['?thing', 'brick:hasPoint', '?sp']],
  0.356574409790115),
 ([['?sen', 'rdf:type', 'brick:Air_Temperature_Sensor'],
   ['?sp', 'rdf:type', 'brick:Supply_Air_Temperature_Setpoint'],
   ['?ahu', 'rdf:type', 'brick:HVAC_Equipment'],
   ['?ahu', 'brick:feeds+', '?thing'],
   ['?thing', 'brick:hasPoint', '?sen'],
   ['?thing', 'brick:hasPoint', '?sp']],
  0.33429329180999484),
 ([['?sen', 'rdf:type', 'brick:Supply_Air_Temperature_Sensor'],
   ['?sp', 'rdf:type', 'brick:Air_Temperature_Setpoint'],
   ['?ahu', 'rdf:type', 'brick:HVAC_Equipment'],
   ['?ahu', 'brick:feeds+', '?thing'],
   ['?thing', 'brick:hasPoint', '?sen'],
   ['?thing', 'brick:hasPoint', '?sp']],
  0.31468400054342793)]

In [21]:
queries_to_print3

[([['?sen', 'rdf:type', 'brick:Air_Temperature_Sensor'],
   ['?sp', 'rdf:type', 'brick:Air_Temperature_Setpoint'],
   ['?ahu', 'rdf:type', 'brick:AHU'],
   ['?ahu', 'brick:feeds+', '?thing'],
   ['?thing', 'brick:hasPoint', '?sen'],
   ['?thing', 'brick:hasPoint', '?sp']],
  0.356574409790115),
 ([['?sen', 'rdf:type', 'brick:Air_Temperature_Sensor'],
   ['?sp', 'rdf:type', 'brick:Air_Temperature_Setpoint'],
   ['?ahu', 'rdf:type', 'brick:HVAC_Equipment'],
   ['?ahu', 'brick:feeds', '?thing'],
   ['?thing', 'brick:hasPoint', '?sen'],
   ['?thing', 'brick:hasPoint', '?sp']],
  0.2738995041741416),
 ([['?sen', 'rdf:type', 'brick:Air_Temperature_Sensor'],
   ['?sp', 'rdf:type', 'brick:Air_Temperature_Setpoint'],
   ['?ahu', 'rdf:type', 'brick:AHU'],
   ['?ahu', 'brick:feeds', '?thing'],
   ['?thing', 'brick:hasPoint', '?sen'],
   ['?thing', 'brick:hasPoint', '?sp']],
  0.1782872048950575),
 ([['?sen', 'rdf:type', 'brick:Air_Temperature_Sensor'],
   ['?sp', 'rdf:type', 'brick:Air_Temperatur